In [1]:
    import os
    import numpy as np
    from tensorflow.keras.models import Sequential, load_model
    from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GRU, TimeDistributed
    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    # Set the path to your dataset
    train_data_dir = r"D:\Downloads\Dataset\train"

    # Specify image dimensions and batch size
    img_width, img_height = 150, 150
    batch_size = 32

    # Extract class names from the folder names
    class_names = sorted(os.listdir(train_data_dir))

    # Create data generator for training data
    train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='sparse',
        shuffle=False)  # Disable shuffling to maintain the order of images

    # Reshape the input data to include a time dimension
    X_train_reshaped = []
    y_train = []

    for i in range(len(train_generator)):
        images, labels = train_generator[i]
        for j in range(len(images)):
            X_train_reshaped.append(images[j])
            y_train.append(labels[j])

    X_train_reshaped = np.array(X_train_reshaped)
    y_train = np.array(y_train, dtype=np.int32)

    # Add a time dimension to the input data
    X_train_reshaped = np.expand_dims(X_train_reshaped, axis=1)

    # Define the GRU model
    model = Sequential()
    model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(1, img_width, img_height, 3)))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu')))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    model.add(TimeDistributed(Flatten()))
    model.add(GRU(128, activation='relu', return_sequences=False))
    model.add(Dense(len(class_names), activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(
        X_train_reshaped,
        y_train,
        batch_size=batch_size,
        epochs=30,
        steps_per_epoch=312)

    # Save the trained model
    model.save('lung_detection_gru.h5')

    # Set the path to your test dataset
    test_data_dir = r"D:\Downloads\Dataset\test"

    # Create data generator for test data
    test_datagen = ImageDataGenerator(rescale=1. / 255)

    test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='sparse',
        shuffle=False)  # Disable shuffling to maintain the order of images

    # Reshape the input data to include a time dimension
    X_test_reshaped = []
    y_test = []

    for i in range(len(test_generator)):
        images, labels = test_generator[i]
        for j in range(len(images)):
            X_test_reshaped.append(images[j])
            y_test.append(labels[j])

    X_test_reshaped = np.array(X_test_reshaped)
    y_test = np.array(y_test, dtype=np.int32)

    # Add a time dimension to the input data
    X_test_reshaped = np.expand_dims(X_test_reshaped, axis=1)

    # Load the trained model
    model = load_model('lung_detection_gru.h5')

    # Evaluate the model on the test data
    loss, accuracy = model.evaluate(X_test_reshaped, y_test)
    print(f"Test loss: {loss}")
    print(f"Test accuracy: {accuracy}")


Found 16050 images belonging to 6 classes.
Epoch 1/30
312/312 [==============================] - 284s 893ms/step - loss: 1.2196 - accuracy: 0.5350
Epoch 2/30
312/312 [==============================] - 282s 906ms/step - loss: 0.7585 - accuracy: 0.7156
Epoch 3/30
312/312 [==============================] - 286s 918ms/step - loss: 0.6256 - accuracy: 0.7670
Epoch 4/30
312/312 [==============================] - 278s 891ms/step - loss: 0.4576 - accuracy: 0.8291
Epoch 5/30
312/312 [==============================] - 278s 892ms/step - loss: 0.3999 - accuracy: 0.8482
Epoch 6/30
312/312 [==============================] - 282s 903ms/step - loss: 0.2723 - accuracy: 0.9013
Epoch 7/30
312/312 [==============================] - 277s 887ms/step - loss: 0.2040 - accuracy: 0.9248
Epoch 8/30
312/312 [==============================] - 274s 879ms/step - loss: 0.1867 - accuracy: 0.9330
Epoch 9/30
312/312 [==============================] - 271s 869ms/step - loss: 0.1115 - accuracy: 0.9618
Epoch 10/30
312/312 [

C:\Users\Hi\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 4110 images belonging to 6 classes.
129/129 [==============================] - 25s 168ms/step - loss: 2.3511 - accuracy: 0.7241
Test loss: 2.3511159420013428
Test accuracy: 0.7240875959396362


In [3]:
# Load the trained model
model = load_model('lung_detection_gru.h5')

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test_reshaped, y_test)
print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")

# Get the raw predictions for the test data
y_pred_probs = model.predict(X_test_reshaped)

# Convert the predicted probabilities to class labels
y_pred = np.argmax(y_pred_probs, axis=1)

# Get the class labels
labels = test_generator.class_indices

# Generate the classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=labels))

# Generate the confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


129/129 [==============================] - 21s 155ms/step - loss: 2.3511 - accuracy: 0.7241
Test loss: 2.3511159420013428
Test accuracy: 0.7240875959396362
129/129 [==============================] - 24s 171ms/step

Classification Report:
              precision    recall  f1-score   support

CARDIOMEGALY       0.78      0.67      0.72       685
       COVID       0.72      0.61      0.66       685
      NORMAL       0.75      0.72      0.74       685
   PNEUMONIA       0.93      0.88      0.91       685
PNEUMOTHORAX       0.49      0.70      0.57       685
TUBERCULOSIS       0.79      0.77      0.78       685

    accuracy                           0.72      4110
   macro avg       0.74      0.72      0.73      4110
weighted avg       0.74      0.72      0.73      4110


Confusion Matrix:
[[457  45   4   2 148  29]
 [ 37 415  34   6 132  61]
 [ 16  27 496  31 108   7]
 [  6  10  39 605  18   7]
 [ 40  52  76   6 477  34]
 [ 31  26   9   0  93 526]]
